In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
#!pip category_encoders
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns; sns.set()
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler
from sklearn import ensemble
from sklearn.metrics import accuracy_score
import scipy.stats as stats
from sklearn.feature_extraction.text import TfidfVectorizer
#import category_encoders as ce
import warnings
import gc
import psutil 
from lightgbm import LGBMClassifier
warnings.filterwarnings('ignore')
%matplotlib inline

os.chdir('/content/drive/My Drive/ML/FinalProject/VEFFinalProject')
os.chdir('predict-stock')
#os.chdir('dataset')
!ls dataset

ls: cannot access 'dataset': No such file or directory


In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [0]:
from datetime import datetime, date
df = pd.read_csv('/content/drive/My Drive/ML/FinalProject/VEFFinalProject/mk-news.csv')
df.time = pd.to_datetime(df.time).dt.date
import time
df = df.loc[df['time']>=date(2009, 1, 1)]
df['returnsClosePrevMktres10'] = df['returnsClosePrevMktres10'].clip(-1,1)

Split data and choose features

# Split data and choose features

In [0]:

def temploral_split(df, label):
    fcol = [c for c in df if c not in ['assetCode', 'assetCodes', 'assetCodesLen', 'assetName', 'audiences', 'binary_returnsNextMktres10','returnsClosePrevMktres10',
                                             'firstCreated', 'headline', 'headlineTag', 'marketCommentary', 'provider', 'cluster_open_close','cluster_volume','cluster_prev_returns', 'open','close',	
                                              'sourceId', 'subjects', 'time_x', 'universe','sourceTimestamp','close_open_diff',
                                             'date','Unnamed 0']]

    df = df[fcol]
    
    #slit 80/20
    train, validate = np.split(df, [int(.7*len(df))])
    
    X_train = train.loc[:, ~train.columns.isin([label])]
    y_train = train[label] > 0
    r_train = train[label]
    time_train = train['time']
    
    X_val = validate.loc[:, ~validate.columns.isin([label])]
    y_val = validate[label] > 0
    r_val = validate[label]
    time_val = validate['time']
    X_train.drop(['time'], axis=1, inplace=True)
    X_val.drop(['time'], axis=1, inplace=True)
    return [X_train, y_train, r_train, time_train, X_val, y_val, r_val, time_val]
    
    return [X_train, y_train, X_val, y_val]
def print_accuracy(model, x, y, x_val, y_val):
    print("Training Accuracy Score: ", accuracy_score(model.predict(x), y))
    print("Validation Accuracy Score: ", accuracy_score(model.predict(x_val), y_val))


In [6]:

X_train_sample, y_train_sample, r_t,time_t, X_val_sample, y_val_sample, r_v, time_v = temploral_split(df, 'returnsOpenNextMktres10')
print(f'X_train_sample: {X_train_sample.shape}')
print(f'X_val_sample: {X_val_sample.shape}')
del df



X_train_sample: (1264223, 50)
X_val_sample: (541811, 50)


# Define Sigma score

In [0]:
# sigma_score function is considered as a custom evaluation metric for xgboost
# example of how custom evaluation function is incorporated into xgboost's training can be found here : https://github.com/dmlc/xgboost/blob/master/demo/guide-python/custom_objective.py
XXX = None
def sigma_score(y_t,r_t,df_t):
    global XXX
    # get y_target values
    labels = r_t
    # call time parameter to be used for grouping, so that we can add x_t values for each day
    df_time = df_t
    
    #calculate x_t and score as specified by the competition
    x_t = pd.Series(y_t*labels)
    x_t_sum = x_t.groupby(df_time).sum()    
    score = (x_t_sum.mean())/(x_t_sum.std())
    XXX = x_t
    return 'sigma_score', round(score,5), labels
    
#y_t as prediction output
def sigma_score_2(y_t,r_t,df_t):
    x_t_i = y_t * r_t
    data = {'day' : df_t, 'x_t_i' : x_t_i}
    df = pd.DataFrame(data)
    x_t = df.groupby('day').sum().values.flatten()
    mean = np.mean(x_t)
    std = np.std(x_t)
    score_valid = mean / std

    return round(score_valid,5)
def evaluation_sigma_score(model,X,up,t):
    y_confidence = model.predict(X)
    y_confidence = y_confidence * 2 - 1
    print('min',y_confidence.min())
    print('max',y_confidence.max())
    print (sigma_score(y_confidence,up,t))

# Train lightgbm model

In [8]:
import lightgbm as lgb
x_1 = [0.07, 2000, 212, 100, 202]
x_2 = [0.1, 2400, 213, 100, 220]

params_1 = {
        'task': 'train',
        'boosting_type': 'dart',
        'objective': 'binary',
        'n_estimators': 500,
        'learning_rate': x_1[0],
        'num_leaves': x_1[1],
        'min_data_in_leaf': x_1[2],
        'num_iteration': x_1[3],
        'max_bin': x_1[4],
        'reg_alpha' : 0.01,
        'reg_lambda' : 0.01,
        'verbose': 1
    }

params_2 = {
        'task': 'train',
        'boosting_type': 'dart',
        'n_estimators': 500,
        'objective': 'binary',
        'learning_rate': x_2[0],
        'num_leaves': x_2[1],
        'min_data_in_leaf': x_2[2],
        'num_iteration': x_2[3],
        'max_bin': x_2[4],
        'reg_alpha' : 0.01,
        'reg_lambda' : 0.01,
        'verbose': 1
    }

lgb1 = lgb.train(params_1, train_set=lgb.Dataset(X_train_sample, label=y_train_sample), num_boost_round=120,
                  valid_sets=[lgb.Dataset(X_val_sample, label=y_val_sample), lgb.Dataset(X_val_sample, label=y_val_sample)],
                  verbose_eval=10, early_stopping_rounds=5)
lgb2 = lgb.train(params_2, train_set=lgb.Dataset(X_train_sample, label=y_train_sample), num_boost_round=120,
                  valid_sets=[lgb.Dataset(X_val_sample, label=y_val_sample), lgb.Dataset(X_val_sample, label=y_val_sample)],
                  verbose_eval=10, early_stopping_rounds=5)

[10]	valid_0's binary_logloss: 0.689932	valid_1's binary_logloss: 0.689932
[20]	valid_0's binary_logloss: 0.692462	valid_1's binary_logloss: 0.692462
[30]	valid_0's binary_logloss: 0.695652	valid_1's binary_logloss: 0.695652
[40]	valid_0's binary_logloss: 0.697581	valid_1's binary_logloss: 0.697581
[50]	valid_0's binary_logloss: 0.697484	valid_1's binary_logloss: 0.697484
[60]	valid_0's binary_logloss: 0.699143	valid_1's binary_logloss: 0.699143
[70]	valid_0's binary_logloss: 0.700077	valid_1's binary_logloss: 0.700077
[80]	valid_0's binary_logloss: 0.701147	valid_1's binary_logloss: 0.701147
[90]	valid_0's binary_logloss: 0.699937	valid_1's binary_logloss: 0.699937
[100]	valid_0's binary_logloss: 0.701401	valid_1's binary_logloss: 0.701401
[10]	valid_0's binary_logloss: 0.691238	valid_1's binary_logloss: 0.691238
[20]	valid_0's binary_logloss: 0.695634	valid_1's binary_logloss: 0.695634
[30]	valid_0's binary_logloss: 0.701213	valid_1's binary_logloss: 0.701213
[40]	valid_0's binary_lo

In [9]:
evaluation_sigma_score(lgb1,X_val_sample,y_val_sample,time_v)

min -0.7242676162326529
max 0.6158921493447158
('sigma_score', -1.15667)


In [10]:
evaluation_sigma_score(lgb2,X_val_sample,y_val_sample,time_v)

min -0.797447022625918
max 0.6580138924129093
('sigma_score', -1.27147)


# Evaluation

In [34]:
y_t = (lgb1.predict(X_val_sample) + lgb2.predict(X_val_sample))/2

y_t = 2*y_t - 1
#print(y_t[:100])
#sigma_score_2(y_t,r_v,time_v)
#a,b,c = sigma_score(y_t,r_v,time_v))
print("test : ",sigma_score(y_t,r_v,time_v))

#y_t = (lgb1.predict(X_train_sample) + lgb2.predict(X_train_sample))/2 
#y_t = 2*y_t - 1
#sigma_score_2(y_t,r_t,time_t)
#print("train : ",sigma_score(y_t,r_t,time_t))

test :  ('sigma_score', 0.35896, 1479022   -0.035415
1479023    0.030965
1479024   -0.124743
1479025    0.017156
1479026   -0.014345
             ...   
2020828   -0.019932
2020829   -0.031742
2020830    0.115330
2020831   -0.045221
2020832   -0.016220
Name: returnsOpenNextMktres10, Length: 541811, dtype: float64)


In [0]:
label = r_v>0 

In [0]:
label = label *1

In [38]:
label_pred = (y_t>0)*1
label_pred

array([0, 0, 0, ..., 0, 0, 1])

In [0]:
from sklearn.metrics import accuracy_score 

In [39]:
accuracy_score(label,label_pred)

0.5258328088577013